In [29]:
# clone the raw_dataset
!git clone https://github.com/jeehuajian/enumeration-for-litghttag-training.git
!pip install botok

fatal: destination path 'enumeration-for-litghttag-training' already exists and is not an empty directory.


In [0]:
import csv
from pathlib import Path
import pdb
import re

import botok
import pandas

In [0]:
data_path = Path('enumeration-for-litghttag-training/etext')

# Helper functions to extract Text with Enumerations

In [0]:
ex_text_fn = data_path/'སོ་སོ་ཐར་པའི་མདོའི་འགྲེལ་པ་རིན་ཆེན་འབྱུང་གནས།.txt'

In [0]:
ex_text = ex_text_fn.read_text()

In [0]:
def preprocess(text):
  text = text.replace('\n', ' ')
  text = text.replace(' ། ', ' །')
  text = text.replace('$','')
  text = text.replace('"', '')
  text = text.replace('¿', '')
  text = text.replace('(E(E', '(E')
  text = re.sub(pattern=r"\d", repl=r"", string=text)
  text = text.replace('\u2028', '')
  return text

In [0]:
ex_text = preprocess(ex_text)

In [0]:
def get_enumeration(text):
  entities = []
  for m in re.finditer(r'\(D[^a-zA-Z]*\)(.*?)\(E[^a-zA-Z].*?\)', text):
    if m.end(0) - m.start(0) < 200:
      entities.append((m.start(0), m.end(0)))
    else:
      for sm in re.finditer(r'(\(E[^a-zA-Z].*?\))', text[m.start(0): m.end(0)]):
        entities.append((m.start(0) + sm.start(0), m.start(0) + sm.end(0)))
                            
  return entities

In [37]:
entities = get_enumeration(ex_text)
for e in entities:
  print(ex_text[e[0]: e[1]])

(Dཕུན་སུམ་ཚོགས་པ་ལྔ)འི་ཚུལ་དུ་བཤད་པ་དང༌། བཤད་ཐབས་ཀྱི་ཡན་ལག་ལྔའི་ཚུལ་དུ་བཤད་པའོ། །དང་པོ་ལ་ལྔ་སྟེ། (Eསྟོན་པ་དང༌། ཆོས་དང༌། དུས་དང༌། གནས་དང༌། འཁོར་)
(Dའཁོར་རྣམ་པ་བཞི་)ནི། (Eཁྱིམ་པའི་ཕྱོགས་ཀྱི་དགེ་བསྙེན་ཕ་མ་གཉིས། རབ་ཏུ་བྱུང་བའི་ཕྱོགས་ཀྱི་དགེ་སློང་ཕ་མ་གཉིས་)
(Dརྫས་སུ་རྣམ་པ་བཞི་)སྟེ། (Eདགེ་ཚུལ། དགེ་སློང༌། དགེ་བསྙེན། བསྙེན་གནས་)
(Eགྲོང་ཁྱེར་གྱི་སྐད་སྨྲ་བ་དང༌། སྙན་ཅིང་འཇེབས་པ་དང༌། རྣམ་པར་གསལ་བ་དང༌། རྣམ་པར་རིག་པ་དང༌། མཉན་པར་འོས་པ་དང༌། མི་མཐུན་པ་མེད་པ་དང༌། བརྟེན་པ་མེད་པ་དང༌། ཕྱིར་མི་གཏུག་པ་)
(Dབཟའ་བ་ལྔ)། (Eཙི་ཙི། ཁྲེ་རྒོད། རྒྱ་སྲན། བྲ་རྒོད། ཁྲེ་)
(Dབཟའ་བར་སྨོན་པ་ལྔ)། (Eགསེར། དངུལ། བཻ་ཌཱུརྻ། ཤེལ། མུ་ཏིག་)
(Dརིན་པོ་ཆེ་ལྔ)། (Eཞ་ཉེ་ཟངས་རོ་ཉེ་ཁྲོ་ཆུ་ལྕགས་)
(Eདཔྱངས་ཚགས། བུམ་ཚགས་གཅེའུ་ཅན། གྲུ་གསུམ་ཚགས། རིལ་བ་ཞབས་ཚགས་ཅག །བསལ་ཚགས་)
(Dཆུ་རུང་བ་ལྔ་)སྟེ། (Eདགེ་འདུན་དང་གང་ཟག་དང༌། ཆུ་རུང་བ་དང༌། ཡིད་ཆེས་པ་དང༌། བརྟགས་)


## Finding context:

**Left context conditions**
- Select still the 2nd whitespace to the left of E, which will include the D.

**Right context conditions**
- Select till the following whitespace

In [0]:
def left_context(match, text, th=2):
  i = match[0]
  while th > 0:
    i -= 1
    if text[i] == ' ': th -= 1
  return i + 1

In [39]:
left_idx = left_context(entities[0], ex_text)
print(ex_text[left_idx: entities[0][1]])

།དང་པོ་ནི། (Dཕུན་སུམ་ཚོགས་པ་ལྔ)འི་ཚུལ་དུ་བཤད་པ་དང༌། བཤད་ཐབས་ཀྱི་ཡན་ལག་ལྔའི་ཚུལ་དུ་བཤད་པའོ། །དང་པོ་ལ་ལྔ་སྟེ། (Eསྟོན་པ་དང༌། ཆོས་དང༌། དུས་དང༌། གནས་དང༌། འཁོར་)


In [0]:
def right_context(match, text, th=2):
  i = match[1]
  while th > 0:
    i += 1
    if text[i] == ' ': th -= 1
  return i - 1

In [41]:
right_idx = right_context(entities[0], ex_text)
print(ex_text[entities[0][0]: right_idx])

(Dཕུན་སུམ་ཚོགས་པ་ལྔ)འི་ཚུལ་དུ་བཤད་པ་དང༌། བཤད་ཐབས་ཀྱི་ཡན་ལག་ལྔའི་ཚུལ་དུ་བཤད་པའོ། །དང་པོ་ལ་ལྔ་སྟེ། (Eསྟོན་པ་དང༌། ཆོས་དང༌། དུས་དང༌། གནས་དང༌། འཁོར་)རོ། །༼བ༽དང་པོ་ལ


In [0]:
def get_with_context(match, text):
  left_idx = left_context(match, text)
  right_idx = right_context(match, text)
  return left_idx, right_idx

In [43]:
idxs = get_with_context(entities[0], ex_text)
print(ex_text[idxs[0]: idxs[1]])

།དང་པོ་ནི། (Dཕུན་སུམ་ཚོགས་པ་ལྔ)འི་ཚུལ་དུ་བཤད་པ་དང༌། བཤད་ཐབས་ཀྱི་ཡན་ལག་ལྔའི་ཚུལ་དུ་བཤད་པའོ། །དང་པོ་ལ་ལྔ་སྟེ། (Eསྟོན་པ་དང༌། ཆོས་དང༌། དུས་དང༌། གནས་དང༌། འཁོར་)རོ། །༼བ༽དང་པོ་ལ


In [44]:
for de in entities:
  idxs = get_with_context(de, ex_text)
  print(ex_text[idxs[0]: idxs[1]])

།དང་པོ་ནི། (Dཕུན་སུམ་ཚོགས་པ་ལྔ)འི་ཚུལ་དུ་བཤད་པ་དང༌། བཤད་ཐབས་ཀྱི་ཡན་ལག་ལྔའི་ཚུལ་དུ་བཤད་པའོ། །དང་པོ་ལ་ལྔ་སྟེ། (Eསྟོན་པ་དང༌། ཆོས་དང༌། དུས་དང༌། གནས་དང༌། འཁོར་)རོ། །༼བ༽དང་པོ་ལ
སོ་ཐར་གྱི་སྡོམ་པ་དེ་ལ་མིང་དུ་དབྱེ་བ་བརྒྱད་དེ། (Dའཁོར་རྣམ་པ་བཞི་)ནི། (Eཁྱིམ་པའི་ཕྱོགས་ཀྱི་དགེ་བསྙེན་ཕ་མ་གཉིས། རབ་ཏུ་བྱུང་བའི་ཕྱོགས་ཀྱི་དགེ་སློང་ཕ་མ་གཉིས་)སོ། །མིང་བསྙེན་གནས་དགེ་བསྙེན་ཕ་མ་གཉིས
དགེ་སློང་ཕ་མ་གཉིས་སོ། །(Dརྫས་སུ་རྣམ་པ་བཞི་)སྟེ། (Eདགེ་ཚུལ། དགེ་སློང༌། དགེ་བསྙེན། བསྙེན་གནས་)ཏེ་བཞི། དགེ་སློང་མཚན་འགྱུར་ན་དགེ་སློང་མ་ཞེས་འཕོ
འཁོར་གྱི་ནང་དུ་ཆོས་སྟོན་པ་ལ་(Dངག་ཡན་ལག་བརྒྱད་)ལྡན་གྱི་འཆད་ཤེས་པ་སྟེ། (Eགྲོང་ཁྱེར་གྱི་སྐད་སྨྲ་བ་དང༌། སྙན་ཅིང་འཇེབས་པ་དང༌། རྣམ་པར་གསལ་བ་དང༌། རྣམ་པར་རིག་པ་དང༌། མཉན་པར་འོས་པ་དང༌། མི་མཐུན་པ་མེད་པ་དང༌། བརྟེན་པ་མེད་པ་དང༌། ཕྱིར་མི་གཏུག་པ་)བརྒྱད་དོ། །གཏོང་བ་སྤེལ་བ་ལ་ཡང་ཅིའི་ཕྱིར་གཏོང་ན་སེམས་ཀྱི་རྒྱུན་མངོན་ཤེས་དྲུག་བསྐྱེད་པའི་ཕྱིར་དང༌
ཉ་ཤ། སྣུམ་ཁུར་)ཏེ་(Dབཟའ་བ་ལྔ)། (Eཙི་ཙི། ཁྲེ་རྒོད། རྒྱ་སྲན། བྲ་རྒོད། ཁྲེ་)སྟེ་(Dབཟའ་བར་སྨོན་པ་ལྔ)། (Eགསེར
བྲ་རྒོད། ཁྲེ་)སྟེ་(Dབཟའ་བར་སྨོན་པ་ལྔ)། (Eགསེར། དངུལ། བཻ་ཌཱུརྻ། ཤེལ

In [0]:
def remove_chars(text, idx, length=1):
  return text[:idx] + text[idx+length:]

def post_processing(text):
  #pdb.set_trace()
  # remove single closing parenthesis
  opening_pren = text.find('(')
  closing_pren = text.find(')')
  if closing_pren < opening_pren:
    text = remove_chars(text, closing_pren)
    
  # remove opening '(E' or '(D'
  rev_text = "".join(reversed(text))
  last_opening_pren = len(text) - rev_text.find('(') - 1
  text = "".join(reversed(rev_text))
  if text[last_opening_pren+1:].find(')') == -1:
    text = remove_chars(text, last_opening_pren, length=2)
  
  # Remove 
  text = text[1:] if text[0] in ['།', ' '] else text
    
  return text

In [46]:
for de in entities:
  idxs = get_with_context(de, ex_text)
  example = post_processing(ex_text[idxs[0]: idxs[1]])
  print(example)

དང་པོ་ནི། (Dཕུན་སུམ་ཚོགས་པ་ལྔ)འི་ཚུལ་དུ་བཤད་པ་དང༌། བཤད་ཐབས་ཀྱི་ཡན་ལག་ལྔའི་ཚུལ་དུ་བཤད་པའོ། །དང་པོ་ལ་ལྔ་སྟེ། (Eསྟོན་པ་དང༌། ཆོས་དང༌། དུས་དང༌། གནས་དང༌། འཁོར་)རོ། །༼བ༽དང་པོ་ལ
སོ་ཐར་གྱི་སྡོམ་པ་དེ་ལ་མིང་དུ་དབྱེ་བ་བརྒྱད་དེ། (Dའཁོར་རྣམ་པ་བཞི་)ནི། (Eཁྱིམ་པའི་ཕྱོགས་ཀྱི་དགེ་བསྙེན་ཕ་མ་གཉིས། རབ་ཏུ་བྱུང་བའི་ཕྱོགས་ཀྱི་དགེ་སློང་ཕ་མ་གཉིས་)སོ། །མིང་བསྙེན་གནས་དགེ་བསྙེན་ཕ་མ་གཉིས
དགེ་སློང་ཕ་མ་གཉིས་སོ། །(Dརྫས་སུ་རྣམ་པ་བཞི་)སྟེ། (Eདགེ་ཚུལ། དགེ་སློང༌། དགེ་བསྙེན། བསྙེན་གནས་)ཏེ་བཞི། དགེ་སློང་མཚན་འགྱུར་ན་དགེ་སློང་མ་ཞེས་འཕོ
འཁོར་གྱི་ནང་དུ་ཆོས་སྟོན་པ་ལ་(Dངག་ཡན་ལག་བརྒྱད་)ལྡན་གྱི་འཆད་ཤེས་པ་སྟེ། (Eགྲོང་ཁྱེར་གྱི་སྐད་སྨྲ་བ་དང༌། སྙན་ཅིང་འཇེབས་པ་དང༌། རྣམ་པར་གསལ་བ་དང༌། རྣམ་པར་རིག་པ་དང༌། མཉན་པར་འོས་པ་དང༌། མི་མཐུན་པ་མེད་པ་དང༌། བརྟེན་པ་མེད་པ་དང༌། ཕྱིར་མི་གཏུག་པ་)བརྒྱད་དོ། །གཏོང་བ་སྤེལ་བ་ལ་ཡང་ཅིའི་ཕྱིར་གཏོང་ན་སེམས་ཀྱི་རྒྱུན་མངོན་ཤེས་དྲུག་བསྐྱེད་པའི་ཕྱིར་དང༌
ཉ་ཤ། སྣུམ་ཁུར་ཏེ་(Dབཟའ་བ་ལྔ)། (Eཙི་ཙི། ཁྲེ་རྒོད། རྒྱ་སྲན། བྲ་རྒོད། ཁྲེ་)སྟེ་(Dབཟའ་བར་སྨོན་པ་ལྔ)། གསེར
བྲ་རྒོད། ཁྲེ་སྟེ་(Dབཟའ་བར་སྨོན་པ་ལྔ)། (Eགསེར། དངུལ། བཻ་ཌཱུརྻ། ཤེལ། མུ་

# Get all the Enumeration examples

In [0]:
examples = []
for fn in data_path.glob('**/*.*'):
  text = fn.read_text()
  text = preprocess(text)
  
  entities = get_enumeration(text)
  for entity in entities:
    idxs = get_with_context(entity, text)
    example = post_processing(text[idxs[0]: idxs[1]])
    examples.append(example)

In [74]:
len(examples)

722

In [75]:
for example in examples[:50]:
  print(example)

(E སྟོང་ཕྲག་བརྒྱ་པ་ཤེར་ཕྱིན་འབུམ་པ།  ཉི་ཁྲི་ལྔ་སྟོང་པ།  ཁྲི་བརྒྱད་སྟོང་པ།  ཤེས་རབ་ཁྲི་པ།  བརྒྱད་སྟོང་པ།  མདོ་སྡུད་པ་སྟེ་ཡུམ་དྲུག་དང༌།  ཤེར་ཕྱིན་བདུན་བརྒྱ་པ།  ལྔ་བརྒྱ་པ།  སུམ་བརྒྱ་པ།  ཚུལ་བརྒྱ་ལྔ་བཅུ་པ།  ལྔ་བཅུ་པ།  སྒོ་ཉེར་ལྔ་པ།  ཀཽའུ་ཤི་ཀས་ཞུས་པ།  རབ་རྩལ་རྣམ་གནོན་གྱིས་ཞུས་པ།  ཤེར་ཕྱིན་ཡི་གེ་ཉུང་ངུུ༌།  ཡི་གེ་གཅིག་པ།  ཤེས་རབ་སྙིང་པོ་སྟེ། སྲས་བཅུ་གཅིག་)བཅས་སོ།། དགོས་པ་ནི
སྟེ། (E འཕགས་པ་རྣམ་གྲོལ་སྡེ།  སློབ་དཔོན་གྲོལ་སྡེ།  ཇོ་བོ་སྨྲྀ་ཏི་ཛྙ་ན།  སེང་བཟང་གིས་ཉི་ཁྲི་དང༌།  བརྒྱད་སྟོང༌།  སྡུད་པ་དང་སྦྱར་བ་དང༌།  བྱམས་པའི་ལུང་བསྟན་ཐོབ་ནས་འགྲེལ་པ་དོན་གསལ།  དེའི་ཡང་འགྲེལ་གསེར་གླིང་པ་ཆོས་གྲགས།  ཡང་དེ་འགྲེལ་ཚིག་གསལ་ཞེས་ཆོས་ཀྱི་བཤེས་གཉེན་གྱིས་མཛད་པ་རྣམས་དང༌།  སྡུད་པ་དང་སྦྱར་བ་སངས་རྒྱས་ཡེ་ཤེས་དང༌།  རྒྱ་འགྲེལ་གྲགས་པའི་ཆ་ཞེས་རཏྣ་ཀིརྟིས་དང༌།  ཤེས་རབ་སྒྲོན་མེའི་ཕྲེང་བ་ཁ་ཆེ་སངས་རྒྱས་དཔལ་གྱིས་མཛད་པ་རྣམས་དང༌།  རིན་འབྱུང་ཞི་བས་མཛད་པའི་དག་ལྡན།  འབུམ་གྱི་རྣམ་བཤད་རྒྱན་སྦྱར་དྷརྨཱ་ཤྲཱིས་མཛད་པ་དང༌།  བརྒྱད་སྟོང་དཀའ་འགྲེལ་སྙིང་པོ་རིན་འབྱུང་ཞི་བ་དང༌།  ཤེར་ཕྱིན་དོན་བསྡུས་སྒྲོན་མ་ཇོ་བོས་མཛད་པ་དང༌།   བརྒྱད་སྟོང་རྒྱན་སྦྱར་གནད

## Formating

```
[
  'example',
  [
    (tag, start, end), 
    (tag, start, end)
  ]
]
```

In [0]:
def remove_anno(example):
  for markup in ['(D', '(E', ')']:
    example = example.replace(markup, '')
  return example

def to_ner_format(example):
  entities = []
  
  en_type = None
  start_idx = None
  i = 0
  ex_i = 0
  while i < len(example) - 1:
    #pdb.set_trace()
    if example[i] == '(':
      if example[i+1] == 'D':
        en_type = 'དབྱེ་གཞི།'  
      else:
        en_type = 'དབྱེ་བ།'
      start_idx = ex_i
    elif example[i] in 'DE':
      pass
    elif example[i] == ')':
      entities.append((en_type, start_idx, ex_i))
    else:
      ex_i += 1
    i += 1
  
  return [remove_anno(example), entities]

In [51]:
test_ex = '012(D345)67(E89)1011'
to_ner_format(test_ex)

['01234567891011', [('དབྱེ་གཞི།', 3, 6), ('དབྱེ་བ།', 8, 10)]]

In [52]:
to_ner_format(examples[0])

['\u2028  སྟོང་ཕྲག་བརྒྱ་པ་ཤེར་ཕྱིན་འབུམ་པ།  ཉི་ཁྲི་ལྔ་སྟོང་པ།  ཁྲི་བརྒྱད་སྟོང་པ། \u2028 ཤེས་རབ་ཁྲི་པ།  བརྒྱད་སྟོང་པ།  མདོ་སྡུད་པ་སྟེ་ཡུམ་དྲུག་དང༌།  ཤེར་ཕྱིན་བདུན་བརྒྱ་པ།  ལྔ་བརྒྱ་པ།  སུམ་བརྒྱ་པ།  ཚུལ་བརྒྱ་ལྔ་བཅུ་པ།  ལྔ་བཅུ་པ།  སྒོ་ཉེར་ལྔ་པ།  ཀཽའུ་ཤི་ཀས་ཞུས་པ།  རབ་རྩལ་རྣམ་གནོན་གྱིས་ཞུས་པ།  ཤེར་ཕྱིན་ཡི་གེ་ཉུང་ངུུ༌།  ཡི་གེ་གཅིག་པ།  ཤེས་རབ་སྙིང་པོ་སྟེ། སྲས་བཅུ་གཅིག་བཅས་སོ།། དགོས་པ་ནི',
 [('དབྱེ་བ།', 2, 352)]]

In [53]:
examples[0].find(')') - 2

352

In [0]:
def ner_formatting(examples):
  ner_format_exs = []
  for example in examples:
    ner_format_exs.append(to_ner_format(example))
  return ner_format_exs

In [76]:
ner_format_exs = ner_formatting(examples)
for e in ner_format_exs[:10]:
  print(e)

[' སྟོང་ཕྲག་བརྒྱ་པ་ཤེར་ཕྱིན་འབུམ་པ།  ཉི་ཁྲི་ལྔ་སྟོང་པ།  ཁྲི་བརྒྱད་སྟོང་པ།  ཤེས་རབ་ཁྲི་པ།  བརྒྱད་སྟོང་པ།  མདོ་སྡུད་པ་སྟེ་ཡུམ་དྲུག་དང༌།  ཤེར་ཕྱིན་བདུན་བརྒྱ་པ།  ལྔ་བརྒྱ་པ།  སུམ་བརྒྱ་པ།  ཚུལ་བརྒྱ་ལྔ་བཅུ་པ།  ལྔ་བཅུ་པ།  སྒོ་ཉེར་ལྔ་པ།  ཀཽའུ་ཤི་ཀས་ཞུས་པ།  རབ་རྩལ་རྣམ་གནོན་གྱིས་ཞུས་པ།  ཤེར་ཕྱིན་ཡི་གེ་ཉུང་ངུུ༌།  ཡི་གེ་གཅིག་པ།  ཤེས་རབ་སྙིང་པོ་སྟེ། སྲས་བཅུ་གཅིག་བཅས་སོ།། དགོས་པ་ནི', [('དབྱེ་བ།', 0, 349)]]
['སྟེ།  འཕགས་པ་རྣམ་གྲོལ་སྡེ།  སློབ་དཔོན་གྲོལ་སྡེ།  ཇོ་བོ་སྨྲྀ་ཏི་ཛྙ་ན།  སེང་བཟང་གིས་ཉི་ཁྲི་དང༌།  བརྒྱད་སྟོང༌།  སྡུད་པ་དང་སྦྱར་བ་དང༌།  བྱམས་པའི་ལུང་བསྟན་ཐོབ་ནས་འགྲེལ་པ་དོན་གསལ།  དེའི་ཡང་འགྲེལ་གསེར་གླིང་པ་ཆོས་གྲགས།  ཡང་དེ་འགྲེལ་ཚིག་གསལ་ཞེས་ཆོས་ཀྱི་བཤེས་གཉེན་གྱིས་མཛད་པ་རྣམས་དང༌།  སྡུད་པ་དང་སྦྱར་བ་སངས་རྒྱས་ཡེ་ཤེས་དང༌།  རྒྱ་འགྲེལ་གྲགས་པའི་ཆ་ཞེས་རཏྣ་ཀིརྟིས་དང༌།  ཤེས་རབ་སྒྲོན་མེའི་ཕྲེང་བ་ཁ་ཆེ་སངས་རྒྱས་དཔལ་གྱིས་མཛད་པ་རྣམས་དང༌།  རིན་འབྱུང་ཞི་བས་མཛད་པའི་དག་ལྡན།  འབུམ་གྱི་རྣམ་བཤད་རྒྱན་སྦྱར་དྷརྨཱ་ཤྲཱིས་མཛད་པ་དང༌།  བརྒྱད་སྟོང་དཀའ་འགྲེལ་སྙིང་པོ་རིན་འབྱུང་ཞི་བ་དང༌།  ཤེར་ཕྱིན་དོན་བསྡུས་སྒྲོན་མ་ཇོ་བོས་མཛད་པ་དང༌།   

# Create NER Dataset
- Tokenize
- Add token level entity tag

In [57]:
wt = botok.tokenizers.wordtokenizer.WordTokenizer()

Building Trie... (16 s.)


In [0]:
def tokenize(sentence):
    result = []
    tokens = wt.tokenize(sentence)
    for token in tokens:
        result.append((token.text, token.pos, token.lemma))
    return result
    
def apply_tag(tags, tokens):
    tags = sorted(tags, key=lambda x: x[1])
    token_last_char_idx = -1
    tag_idx = 0
    is_begining = True
    
    result = []
    for token in tokens:
        token_last_char_idx += len(token[0])
        tag = tags[tag_idx]
        if token_last_char_idx > tag[1] and token_last_char_idx <= tag[2]:
            if is_begining:
                result.append(f'B-{tag[0]}')
                is_begining = False
            else:
                result.append(f'I-{tag[0]}')
        else:
            result.append('O')
            is_begining = True
            if len(result) > 2 and result[-2][0] in ['B', 'I'] and tag_idx < len(tags) - 1: 
                tag_idx += 1
    
    return result

def get_tokens_with_ner_tag(example):
    sentence = example[0]
    tags = example[1]

    tokens = tokenize(sentence)
    tokens_tag = apply_tag(tags, tokens)
    
    return tokens, tokens_tag

In [85]:
idx = 8
print(examples[idx])
get_tokens_with_ner_tag(ner_format_exs[idx])

མཚན་ཉིད་མེད་པའི་རྣམ་པ་སྟེ་(Dསྡུག་བསྔལ་གྱི་བདེན་པའི་རྣམ་པ་ལྔ་)དང༌། (Eནང་དང༌། ཕྱི་དང༌། གཉིས་ཀ་དང༌། སྟོང་པ་དང༌། ཆེན་པོ་དང༌། དོན་དམ་པ་དང༌། འདུས་བྱས་དང༌། འདུས་མ་བྱས་དང༌། མཐའ་ལས་འདས་པ་དང༌། ཐོག་མ་དང་ཐ་མ་མེད་པ་དང༌། དོར་བ་མེད་པ་དང༌། རང་བཞིན་དང༌། ཆོས་ཐམས་ཅད་དང༌། རང་གི་མཚན་ཉིད་དང༌། མི་དམིགས་པ་སྟོང་པ་ཉིད་དང༌། དངོས་པོ་མེད་པའི་ངོ་བོ་ཉིད་སྟོང་པ་ཉིད་)བཅས་སྟེ་(Dའགོག་པའི་བདེན་པའི་རྣམ་པ་བཅུ་དྲུག་)སྟེ་ལམ་ཤེས་ཀྱི་རྣམ་པ་སོ་དྲུག་ཡིན་ནོ།། གསུམ་པ་རྣམ་མཁྱེན་གྱི་རྣམ་པ་ནི


([('མཚན་ཉིད་', 'NOUN', 'མཚན་ཉིད་'),
  ('མེད་པ', 'VERB', 'མེད་པ་'),
  ('འི་', 'PART', 'གི་'),
  ('རྣམ་པ་', 'NOUN', 'རྣམ་པ་'),
  ('སྟེ་', 'ADP', 'སྟེ་'),
  ('སྡུག་བསྔལ་', 'NOUN', 'སྡུག་བསྔལ་'),
  ('གྱི་', 'ADP', 'གྱི་'),
  ('བདེན་པ', 'VERB', 'བདེན་པ་'),
  ('འི་', 'PART', 'གི་'),
  ('རྣམ་པ་', 'NOUN', 'རྣམ་པ་'),
  ('ལྔ་', 'NUM', 'ལྔ་'),
  ('དང༌', 'ADP', 'དང་'),
  ('། ', '', ''),
  ('ནང་', 'NOUN', 'ནང་'),
  ('དང༌', 'ADP', 'དང་'),
  ('། ', '', ''),
  ('ཕྱི་', 'NOUN', 'ཕྱི་'),
  ('དང༌', 'ADP', 'དང་'),
  ('། ', '', ''),
  ('གཉིས་ཀ་', 'OTHER', 'གཉིས་ཀ་'),
  ('དང༌', 'ADP', 'དང་'),
  ('། ', '', ''),
  ('སྟོང་པ་', 'VERB', 'སྟོང་པ་'),
  ('དང༌', 'ADP', 'དང་'),
  ('། ', '', ''),
  ('ཆེན་པོ་', 'ADJ', 'ཆེན་པོ་'),
  ('དང༌', 'ADP', 'དང་'),
  ('། ', '', ''),
  ('དོན་དམ་པ་', 'NOUN', None),
  ('དང༌', 'ADP', 'དང་'),
  ('། ', '', ''),
  ('འདུས་བྱས་', 'NOUN', 'འདུས་བྱས་'),
  ('དང༌', 'ADP', 'དང་'),
  ('། ', '', ''),
  ('འདུས་མ་བྱས་', 'OOV', 'འདུས་མ་བྱས་'),
  ('དང༌', 'ADP', 'དང་'),
  ('། ', '', ''),
  ('མཐའ་', '

### Save to CSV file

In [0]:
def save_to_csv(file, ds):
    with file.open('w') as csv_file:
        fieldnames = ['sentence_idx', 'word', 'tag', 'pos', 'lemma']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        
        for i, d in enumerate(ds):
            for token, tag in zip(*get_tokens_with_ner_tag(d)):
                writer.writerow({
                    'sentence_idx': i+1, 
                    'word': token[0],
                    'tag': tag,
                    'pos': token[1],
                    'lemma': token[2]
                })

In [0]:
output_file = Path(f'enumerations-{len(examples)}.csv')
save_to_csv(output_file, ner_format_exs)

In [0]:
df = pandas.read_csv(output_file)

In [92]:
df.head(60)

,sentence_idx,word,tag,pos,lemma
0,1,སྟོང་ཕྲག་,B-དབྱེ་བ།,OTHER,སྟོང་ཕྲག་
1,1,བརྒྱ་པ་,I-དབྱེ་བ།,NUM,NaN
2,1,ཤེར་ཕྱིན་,I-དབྱེ་བ།,PROPN,ཤེར་ཕྱིན་
3,1,འབུམ་པ,I-དབྱེ་བ།,NOUN,འབུམ་པ་
4,1,།,I-དབྱེ་བ།,NaN,NaN
5,1,ཉི་ཁྲི་,I-དབྱེ་བ།,OTHER,ཉི་ཁྲི་
6,1,ལྔ་,I-དབྱེ་བ།,NUM,ལྔ་
7,1,སྟོང་པ,I-དབྱེ་བ།,VERB,སྟོང་པ་
8,1,།,I-དབྱེ་བ།,NaN,NaN
9,1,ཁྲི་,I-དབྱེ་བ།,NUM,ཁྲི་
